In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.cuda.is_available()

True

In [2]:
from src.vlstm_v4.interface import qkvkernel

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/vlstm_v4/build.ninja...
Building extension module vlstm_v4...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module vlstm_v4...


### qkvtest

In [9]:
S = 16 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [18]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 100.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 10.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 10.
qs, qs.shape

(tensor([[[[0.0000, 0.0100, 0.0200, 0.0300],
           [0.0400, 0.0500, 0.0601, 0.0698],
           [0.0801, 0.0898, 0.1001, 0.1099],
           [0.1201, 0.1299, 0.1396, 0.1504],
           [0.1602, 0.1699, 0.1797, 0.1904],
           [0.2002, 0.2100, 0.2197, 0.2305],
           [0.2402, 0.2500, 0.2598, 0.2695],
           [0.2793, 0.2891, 0.3008, 0.3105],
           [0.3203, 0.3301, 0.3398, 0.3496],
           [0.3594, 0.3691, 0.3809, 0.3906],
           [0.4004, 0.4102, 0.4199, 0.4297],
           [0.4395, 0.4492, 0.4609, 0.4707],
           [0.4805, 0.4902, 0.5000, 0.5117],
           [0.5195, 0.5312, 0.5391, 0.5508],
           [0.5586, 0.5703, 0.5781, 0.5898],
           [0.6016, 0.6094, 0.6211, 0.6289]]]], device='cuda:0', dtype=torch.bfloat16),
 torch.Size([1, 1, 16, 4]))

In [19]:
torch.set_printoptions(linewidth=200)
rs = qs @ ks.transpose(-1, -2)
rs, rs.shape

(tensor([[[[0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060],
           [0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220],
           [0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381],
           [0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540, 0.0540],
           [0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703, 0.0703],
           [0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859, 0.0859],
           [0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021, 0.1021],
           [0

In [20]:
# pytorch version
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[0.0096, 0.0096, 0.0096, 0.0096],
           [0.0352, 0.0352, 0.0352, 0.0352],
           [0.0610, 0.0610, 0.0610, 0.0610],
           [0.0864, 0.0864, 0.0864, 0.0864],
           [0.1128, 0.1128, 0.1128, 0.1128],
           [0.1377, 0.1377, 0.1377, 0.1377],
           [0.1631, 0.1631, 0.1631, 0.1631],
           [0.1895, 0.1895, 0.1895, 0.1895],
           [0.2139, 0.2139, 0.2139, 0.2139],
           [0.2412, 0.2412, 0.2412, 0.2412],
           [0.2656, 0.2656, 0.2656, 0.2656],
           [0.2930, 0.2930, 0.2930, 0.2930],
           [0.3184, 0.3184, 0.3184, 0.3184],
           [0.3418, 0.3418, 0.3418, 0.3418],
           [0.3672, 0.3672, 0.3672, 0.3672],
           [0.3945, 0.3945, 0.3945, 0.3945]]]], device='cuda:0', dtype=torch.bfloat16),
 torch.Size([1, 1, 16, 4]))

In [21]:
# cuda kernel
rs = qkvkernel(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

before kernel dispatch - bfloat16!
B: 1, NH: 1, S: 16, DH: 4
blocksxy: 1-1, threads: 4-4
In Kernel: gdim.x: 1, gdim.y: 1, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
qTIdx=0|kvTIdx=0: qTile[3][3] = 0.150391
qTIdx=0|kvTIdx=0: kTile[0][3] = 0.100098
qTIdx=0|kvTIdx=0: sTile[3][0](3) = 0.054077
qTIdx=0|kvTIdx=0: qTile[7][3] = 0.310547
qTIdx=0|kvTIdx=0: kTile[0][3] = 0.100098
qTIdx=0|kvTIdx=0: sTile[7][0](3) = 0.118164
qTIdx=1|kvTIdx=0: qTile[3][3] = 0.470703
qTIdx=1|kvTIdx=0: kTile[0][3] = 0.100098
qTIdx=1|kvTIdx=0: sTile[3][0](3) = 0.182129
qTIdx=1|kvTIdx=0: qTile[7][3] = 0.628906
qTIdx=1|kvTIdx=0: kTile[0][3] = 0.100098
qTIdx=1|kvTIdx=0: sTile[7][0](3) = 0.246582


(tensor([[[[0.0096, 0.0096, 0.0096, 0.0096],
           [0.0352, 0.0352, 0.0352, 0.0352],
           [0.0610, 0.0610, 0.0610, 0.0610],
           [0.0869, 0.0869, 0.0869, 0.0869],
           [0.1118, 0.1118, 0.1118, 0.1118],
           [0.1377, 0.1377, 0.1377, 0.1377],
           [0.1631, 0.1631, 0.1631, 0.1631],
           [0.1895, 0.1895, 0.1895, 0.1895],
           [0.2236, 0.2236, 0.2236, 0.2236],
           [0.2773, 0.2773, 0.2773, 0.2773],
           [0.3262, 0.3262, 0.3262, 0.3262],
           [0.3789, 0.3789, 0.3789, 0.3789],
           [0.4297, 0.4297, 0.4297, 0.4297],
           [0.4805, 0.4805, 0.4805, 0.4805],
           [0.5312, 0.5312, 0.5312, 0.5312],
           [0.5859, 0.5859, 0.5859, 0.5859]]]], device='cuda:0', dtype=torch.bfloat16),
 torch.Size([1, 1, 16, 4]))

### Matmul kernel Test from vlstm_v2

In [10]:
# from src.vlstm_v3.interface import testkernel, copykernel, mmkernelv1

In [7]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [12]:
S = 8 # sequence length
DH = 8 # hidden size

In [13]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape, matA, matA.sum(-1)

(torch.Size([16, 8]),
 torch.Size([8, 8]),
 tensor([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
         [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
         [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.],
         [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
         [ 32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
         [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.],
         [ 48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.],
         [ 56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.],
         [ 64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.],
         [ 72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
         [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.],
         [ 88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.],
         [ 96.,  97.,  98.,  99., 100., 101., 102., 103.],
         [104., 105., 106., 107., 108., 109., 110., 111.],
         [112., 113., 114., 115., 116., 117., 118., 119.],
         [120., 121., 122., 123., 124., 125., 126., 127.]], device='cuda

In [14]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[ 28.,  28.,  28.,  28.,  28.,  28.,  28.,  28.],
         [ 92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.],
         [156., 156., 156., 156., 156., 156., 156., 156.],
         [220., 220., 220., 220., 220., 220., 220., 220.],
         [284., 284., 284., 284., 284., 284., 284., 284.],
         [348., 348., 348., 348., 348., 348., 348., 348.],
         [412., 412., 412., 412., 412., 412., 412., 412.],
         [476., 476., 476., 476., 476., 476., 476., 476.],
         [540., 540., 540., 540., 540., 540., 540., 540.],
         [604., 604., 604., 604., 604., 604., 604., 604.],
         [668., 668., 668., 668., 668., 668., 668., 668.],
         [732., 732., 732., 732., 732., 732., 732., 732.],
         [796., 796., 796., 796., 796., 796., 796., 796.],
         [860., 860., 860., 860., 860., 860., 860., 860.],
         [924., 924., 924., 924., 924., 924., 924., 924.],
         [988., 988., 988., 988., 988., 988., 988., 988.]], device='cuda:0', dtype=torch.bfloat16),
 torch.Size([16

In [15]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [16]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

NameError: name 'mmkernelv1' is not defined

In [ ]:
matA[9]

In [ ]:
matA[9].cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cuda:0')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float32, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float16, device=torch.device('cuda:0')).cumsum(-1)

In [8]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [ ]:
# mat @ mat.T @ mat

### pytorch version

In [ ]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [ ]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))